Step 1: Create conda environment

In [ ]:
%%bash
# Some users experienced issues on Ubuntu with an AMD CPU
# Install libopenblas-dev (issue #115, thanks WindWing)
# sudo apt-get install libopenblas-dev
export TORCH_CUDA_ARCH_LIST="6.0 6.1 6.2 7.0 7.2 7.5 8.0 8.6"
conda env create -f environment_small.yml
conda activate mask_3d

Step 2: Install cuda-toolkit and correct gxx version

In [ ]:
%%bash
#conda install nvidia/label/cuda-11.3.1::cuda-toolkit
conda install conda-forge::cudatoolkit=11.3.1
conda install conda-forge::'gxx[version=">=5,<10.2.1"]'

Step 3: Install pip requirements

In [ ]:
%%bash
pip install pip==23.3
PIP_CONSTRAINT=constraints.txt pip install -r requirements.txt --no-deps --no-cache

Step 4: Install pyviz3d

In [ ]:
%pip install pyviz3d==0.2.28

Step 5: Install torch, torch scatter and detectron2

In [ ]:
pip3 install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
pip3 install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
#pip3 install 'git+https://github.com/facebookresearch/detectron2.git@710e7795d0eeadf9def0e7ef957eea13532e34cf' --no-deps
cd third_party
git clone https://github.com/facebookresearch/detectron2.git
python -m pip install -e detectron2

Step 6: Install the MinkowskiEngine

In [ ]:
%%bash
git clone --recursive "https://github.com/NVIDIA/MinkowskiEngine"
cd MinkowskiEngine
conda install -c conda-forge blas=*=openblas
git checkout 02fc608bea4c0549b0a7b00ca1bf15dee4a0b228
python setup.py install --force_cuda --blas=openblas

Step 7: Setup ScanNet

In [ ]:
%%bash
cd ..
git clone https://github.com/ScanNet/ScanNet.git
cd ScanNet/Segmentator
git checkout 3e5726500896748521a6ceb81271b0f5b2c0e7d2
make

Step 8: Setup PointNet2

In [ ]:
%%bash
cd ../../pointnet2
python setup.py install

Step 9: Install pytorch-lightning

In [ ]:
%%bash
cd ../../
pip3 install pytorch-lightning==1.7.2 --no-deps

If your unable to run the model after step 9, check the version of hydra-core and omegaconf and update them

In [ ]:
%%bash
pip install hydra-core==1.0.5
pip install omegaconf==2.2

# 2. Datasets

## Semantic Terrain Points Labeling - Synthetic 3D (STPLS3D)

Download from: https://drive.google.com/file/d/1ZGS5WDL3th7_OsC7E4kAfvOXjCY9ugyR/view

Extract and put content into path: data/raw/stpls3d

Split into train and validation dataset (data/raw/stpls3d/train and data/raw/stpls3d/validation): 
- trainSplit = [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 21, 22, 23, 24]    
- valSplit = [5, 10, 15, 20, 25] 

In [12]:
!python -m datasets.preprocessing.stpls3d_preprocessing preprocess --data_dir="data/raw/stpls3d" --save_dir="data/processed/stpls3d"

2025-01-18 16:55:42.638 | INFO     | datasets.preprocessing.base_preprocessing:preprocess:47 - Tasks for train: 20
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   4 out of  20 | elapsed:   50.8s remaining:  3.4min
[Parallel(n_jobs=16)]: Done   7 out of  20 | elapsed:   55.7s remaining:  1.7min
[Parallel(n_jobs=16)]: Done  10 out of  20 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=16)]: Done  13 out of  20 | elapsed:  1.2min remaining:   38.7s
[Parallel(n_jobs=16)]: Done  16 out of  20 | elapsed:  1.3min remaining:   18.8s
[Parallel(n_jobs=16)]: Done  20 out of  20 | elapsed:  1.4min finished
2025-01-18 16:57:08.306 | INFO     | datasets.preprocessing.base_preprocessing:preprocess:47 - Tasks for validation: 5
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of   5 | elapsed:   38.3s remaining:   57.4s
[Parallel(n_jobs=16)]: Done   3 out of   5 | elapsed:  

# 3. Training and testing

## Semantic Terrain Points Labeling - Synthetic 3D (STPLS3D)

Download checkpoint from Github Repo and extract into path "checkpoints/stpls3d/stpls3d_val.ckpt"

In [13]:
!sh scripts/stpls3d/stpls3d_val.sh

/rhome/blessman/miniconda3/envs/mask3d/lib/python3.10/site-packages/pytorch_lightning/utilities/seed.py:55: UserWarning: No seed found, seed set to 1436322102
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1436322102
EXPERIMENT ALREADY EXIST
{'_target_': 'pytorch_lightning.loggers.WandbLogger', 'project': '${general.project_name}', 'name': '${general.experiment_name}', 'save_dir': '${general.save_dir}', 'entity': 'schult', 'resume': 'allow', 'id': '${general.experiment_name}'}
{'_target_': 'models.Res16UNet18B', 'config': {'dialations': [1, 1, 1, 1], 'conv1_kernel_size': 5, 'bn_momentum': 0.02}, 'in_channels': '${data.in_channels}', 'out_channels': '${data.num_labels}', 'out_fpn': True}
[2025-01-18 16:59:05,130][__main__][INFO] - {'general_train_mode': False, 'general_task': 'instance_segmentation', 'general_seed': None, 'general_checkpoint': 'checkpoints/stpls3d/stpls3d_benchmark_03.ckpt', 'general_backbone_checkpoint': None, 'general_freeze_backbone': False

/rhome/blessman/miniconda3/envs/mask3d/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:52: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v1.7. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
/rhome/blessman/miniconda3/envs/mask3d/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:57: LightningDeprecationWarning: Setting `Trainer(weights_save_path=)` has been deprecated in v1.6 and will be removed in v1.8. Please pass ``dirpath`` directly to the `ModelCheckpoint` callback
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cluster/54/blessman/ml3d/models/Mask3D/datasets/semseg.py:696: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsa

Testing DataLoader 0:   3%|▍                 | 13/500 [04:55<3:04:41, 22.75s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:   3%|▌                 | 14/500 [05:13<3:01:06, 22.36s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

Testing DataLoader 0:   6%|█                 | 28/500 [12:41<3:33:49, 27.18s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:   6%|█                 | 29/500 [13:01<3:31:26, 26.94s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

Testing DataLoader 0:   9%|█▌                | 43/500 [19:48<3:30:33, 27.65s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:   9%|█▌                | 44/500 [19:59<3:27:09, 27.26s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

...

Testing DataLoader 0:  75%|████████████▋    | 373/500 [1:33:42<31:54, 15.07s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:  75%|████████████▋    | 374/500 [1:34:22<31:47, 15.14s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

Testing DataLoader 0:  78%|█████████████▏   | 388/500 [1:36:50<27:57, 14.98s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:  78%|█████████████▏   | 389/500 [1:36:57<27:39, 14.95s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

Testing DataLoader 0:  81%|█████████████▋   | 403/500 [1:38:48<23:46, 14.71s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:  81%|█████████████▋   | 404/500 [1:38:53<23:30, 14.69s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

Testing DataLoader 0:  84%|██████████████▏  | 418/500 [1:40:47<19:46, 14.47s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:  84%|██████████████▏  | 419/500 [1:40:52<19:30, 14.44s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode

Testing DataLoader 0:  87%|██████████████▋  | 433/500 [1:42:52<15:55, 14.26s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_indices = topk_indices // num_classes
Testing DataLoader 0:  87%|██████████████▊  | 434/500 [1:43:22<15:43, 14.29s/it]/cluster/54/blessman/ml3d/models/Mask3D/trainer/trainer.py:599: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode